In [2]:
import pandas as pd

# Define correct file paths (update if necessary)
file1 = "Data/Police_Department_Incident_Reports__Historical_2003_to_May_2018.csv"
file2 = "Data/Police_Department_Incident_Reports__2018_to_Present.csv"

# Load datasets
df1 = pd.read_csv(file1, low_memory=False)
df2 = pd.read_csv(file2, low_memory=False)

# Check first few rows
print(df1.head())
print(df2.head())


             PdId  IncidntNum  Incident Code       Category  \
0   4133422003074    41334220           3074        ROBBERY   
1   5118535807021    51185358           7021  VEHICLE THEFT   
2   4018830907021    40188309           7021  VEHICLE THEFT   
3  11014543126030   110145431          26030          ARSON   
4  10108108004134   101081080           4134        ASSAULT   

                Descript DayOfWeek        Date   Time PdDistrict Resolution  \
0  ROBBERY, BODILY FORCE    Monday  11/22/2004  17:50  INGLESIDE       NONE   
1      STOLEN AUTOMOBILE   Tuesday  10/18/2005  20:00       PARK       NONE   
2      STOLEN AUTOMOBILE    Sunday  02/15/2004  02:00   SOUTHERN       NONE   
3                  ARSON    Friday  02/18/2011  05:27  INGLESIDE       NONE   
4                BATTERY    Sunday  11/21/2010  17:00   SOUTHERN       NONE   

   ... Fix It Zones as of 2017-11-06  2 2  DELETE - HSOC Zones 2 2  \
0  ...                                NaN                      NaN   
1  ...

In [25]:
import pandas as pd
import os

# Define correct file paths
file1 = "Data/Police_Department_Incident_Reports__Historical_2003_to_May_2018.csv"
file2 = "Data/Police_Department_Incident_Reports__2018_to_Present.csv"

# ✅ Load datasets
df1 = pd.read_csv(file1, low_memory=False)
df2 = pd.read_csv(file2, low_memory=False)

# ✅ Step 1: Remove Duplicate Columns in df2
df2 = df2.loc[:, ~df2.columns.duplicated()]

# ✅ Step 2: Trim Column Names (Remove Extra Spaces)
df1.columns = df1.columns.str.strip()
df2.columns = df2.columns.str.strip()

# ✅ Step 3: Rename Columns for Consistency
df1.rename(columns={
    "Date": "Incident Date",
    "Time": "Incident Time",
    "DayOfWeek": "Incident Day of Week",
    "Category": "Incident Category",
    "Descript": "Incident Description",
    "PdDistrict": "Police District",
    "X": "Longitude",
    "Y": "Latitude"
}, inplace=True)

df2.rename(columns={"Incident Subcategory": "Incident Description"}, inplace=True)

# ✅ Step 4: Keep Only Relevant Columns
columns_to_keep = ["Incident Date", "Incident Time", "Incident Day of Week",
                   "Incident Category", "Incident Description",
                   "Police District", "Latitude", "Longitude"]

df1 = df1[[col for col in columns_to_keep if col in df1.columns]]
df2 = df2[[col for col in columns_to_keep if col in df2.columns]]

print("✅ Only relevant columns kept.")

# ✅ Step 5: Convert 'Incident Date' to datetime format
df1["Incident Date"] = pd.to_datetime(df1["Incident Date"], errors="coerce")
df2["Incident Date"] = pd.to_datetime(df2["Incident Date"], errors="coerce")

# ✅ Step 6: Filter only Full Years
df1 = df1[(df1["Incident Date"] >= "2003-01-01") & (df1["Incident Date"] <= "2017-12-31")]
df2 = df2[(df2["Incident Date"] >= "2018-01-01") & (df2["Incident Date"] <= "2023-12-31")]

print("✅ Dates converted and filtered!")

# ✅ Step 7: Ensure Columns Are Exactly the Same Before Merging
missing_cols_df1 = set(df2.columns) - set(df1.columns)
missing_cols_df2 = set(df1.columns) - set(df2.columns)

for col in missing_cols_df1:
    df1[col] = None  # Add missing columns to df1

for col in missing_cols_df2:
    df2[col] = None  # Add missing columns to df2

# ✅ Step 8: Reset Index to Avoid Merge Errors
df1.reset_index(drop=True, inplace=True)
df2.reset_index(drop=True, inplace=True)
df2 = df2.loc[:, ~df2.columns.duplicated()]


print("df1 data types:\n", df1.dtypes)
print("df2 data types:\n", df2.dtypes)


# ✅ Step 9: Merge the Cleaned Datasets
df_final = pd.concat([df1, df2], ignore_index=True)
print(f"✅ Datasets merged successfully! Total rows: {df_final.shape[0]}")

# ✅ Step 10: Standardize Crime Type Names
df_final["Incident Category"] = df_final["Incident Category"].str.upper().str.strip()
df_final["Incident Description"] = df_final["Incident Description"].str.upper().str.strip()

# ✅ Step 11: Remove Duplicates
before_dedup = df_final.shape[0]
df_final.drop_duplicates(inplace=True)
after_dedup = df_final.shape[0]
print(f"✅ Removed {before_dedup - after_dedup} duplicate rows.")

# ✅ Step 12: Handle Missing Values
df_final.dropna(subset=["Incident Category", "Police District"], inplace=True)
df_final["Latitude"].fillna(df_final["Latitude"].median(), inplace=True)
df_final["Longitude"].fillna(df_final["Longitude"].median(), inplace=True)

print(f"✅ Missing values handled. Total remaining rows: {df_final.shape[0]}")

# ✅ Step 13: Save Cleaned Dataset
save_path = "Data/SF_Crime_Data_Cleaned.csv"
os.makedirs(os.path.dirname(save_path), exist_ok=True)
df_final.to_csv(save_path, index=False)

print("🎉 Final cleaned dataset saved successfully!")


✅ Only relevant columns kept.
✅ Dates converted and filtered!
df1 data types:
 Incident Date           datetime64[ns]
Incident Time                   object
Incident Day of Week            object
Incident Category               object
Incident Description            object
Police District                 object
Latitude                       float64
Longitude                      float64
dtype: object
df2 data types:
 Incident Date           datetime64[ns]
Incident Time                   object
Incident Day of Week            object
Incident Category               object
Incident Description            object
Police District                 object
Latitude                       float64
Longitude                      float64
dtype: object
✅ Datasets merged successfully! Total rows: 2897375
✅ Removed 40373 duplicate rows.


/var/folders/lr/_ynnd4hs17b6vnnx6lbz3hzh0000gn/T/ipykernel_22196/2153475292.py:89: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_final["Latitude"].fillna(df_final["Latitude"].median(), inplace=True)
/var/folders/lr/_ynnd4hs17b6vnnx6lbz3hzh0000gn/T/ipykernel_22196/2153475292.py:90: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which 

✅ Missing values handled. Total remaining rows: 2856300
🎉 Final cleaned dataset saved successfully!
